In [25]:
# @title Setup
from google.cloud import bigquery
from google.colab import data_table
import bigframes.pandas as bpd
import pandas as pd
from google.colab import files

project = 'pnad-covid-460322' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

# CONSULTA PNAD

In [26]:
job = client.get_job('bquxjob_2d98cf22_197075cb7f8')
destination = job.destination
bq_df = bpd.read_gbq(f"{destination.project}.{destination.dataset_id}.{destination.table_id}")
pandas_df = bq_df.to_pandas()

pandas_df

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2025-05-25 12:14:19.154229+00:00 to avoid
incompatibilies with previous reads of this table. To read the latest
version, set `use_cache=False` or close the current session with
Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


,UF,mes_pesquisa,idade,sexo,escolaridade,recebeu_auxilio,teve_febre_semana_passada,teve_tosse_semana_passada,teve_dor_garganta_semana_passada,teve_dificuldade_respirar,...,teve_nausea,teve_nariz_entupido,trabalhou_semana_passada,fez_exame,resultado_swab,resultado_furo_no_dedo,resultado_veia_no_braco,providencia_exame,resultado_final,total
0,21,9,53,1,5,2,2,2,2,1,...,2,2,2.0,1,<NA>,2.0,<NA>,3,Negativo,1
1,28,7,37,1,3,1,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,1,<NA>,1
2,29,8,82,1,2,2,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,3,<NA>,4
3,35,7,54,1,2,1,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,4,<NA>,2
4,29,8,7,2,1,1,2,2,2,2,...,2,2,<NA>,2,<NA>,<NA>,<NA>,4,<NA>,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473238,21,7,24,1,4,2,2,2,2,2,...,2,2,1.0,2,<NA>,<NA>,<NA>,2,<NA>,1
473239,26,9,19,2,4,1,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,4,<NA>,1
473240,43,7,21,1,2,1,2,2,2,2,...,2,2,1.0,2,<NA>,<NA>,<NA>,3,<NA>,1
473241,53,7,70,2,3,2,2,2,2,2,...,2,2,2.0,2,<NA>,<NA>,<NA>,4,<NA>,2


# DICIONÁRIO DE RESPOSTA


In [27]:
job_dict = client.get_job('bquxjob_26fb41c7_197075c4f54')
destination_dict = job_dict.destination
bq_dict_df = bpd.read_gbq(f"{destination_dict.project}.{destination_dict.dataset_id}.{destination_dict.table_id}")
pandas_dict_df = bq_dict_df.to_pandas()

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: UserWarning: Reading cached table from 2025-05-25 12:14:34.158734+00:00 to avoid
incompatibilies with previous reads of this table. To read the latest
version, set `use_cache=False` or close the current session with
Session.close() or bigframes.pandas.close_session().
  exec(code_obj, self.user_global_ns, self.user_ns)


# TRATAMENTO DE RESPOSTAS

In [28]:
df_query = pandas_df.copy()

dict_map = {}
for question in pandas_dict_df['Question'].unique():
    sub_df = pandas_dict_df[pandas_dict_df['Question'] == question]
    mapping = dict(zip(sub_df['Value'], sub_df['Ref']))
    dict_map[question] = mapping

# Agora aplicamos esse mapeamento para cada coluna da query:
for col in df_query.columns:
    if col in dict_map:
        df_query[col] = df_query[col].map(dict_map[col])


### **ANÁLISE TEMPORAL**
* Quantidade de testes
* Avaliar tipo de teste
* Providencias tomadas
* Trabalhou na semana anterior
* Recebeu auxílio

In [29]:
df_query_temp = df_query.groupby(['mes_pesquisa'], as_index=False).agg({'total':'sum'})
df_query_temp = df_query_temp.pivot_table(index='mes_pesquisa', values='total')


df_query_temp_exam = df_query.groupby(['mes_pesquisa', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_temp_exam = df_query_temp_exam.pivot_table(index='mes_pesquisa', columns='fez_exame', values='total')

df_query_temp_result = df_query.groupby(['mes_pesquisa', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_temp_result = df_query_temp_result.pivot_table(index='mes_pesquisa', columns='resultado_final', values='total')


df_query_temp_trab = df_query.groupby(['mes_pesquisa', 'trabalhou_semana_passada'], as_index=False).agg({'total':'sum'})
df_query_temp_trab = df_query_temp_trab.pivot_table(index='mes_pesquisa', columns='trabalhou_semana_passada', values='total')

df_query_temp_aux = df_query.groupby(['mes_pesquisa', 'recebeu_auxilio'], as_index=False).agg({'total':'sum'})
df_query_temp_aux = df_query_temp_aux.pivot_table(index='mes_pesquisa', columns='recebeu_auxilio', values='total')

with pd.ExcelWriter("Temporal.xlsx", engine="openpyxl") as writer:
    df_query_temp.to_excel(writer, sheet_name="Total por Mês")
    df_query_temp_exam.to_excel(writer, sheet_name="Fez Exame")
    df_query_temp_result.to_excel(writer, sheet_name="Resultados Exame")
    df_query_temp_trab.to_excel(writer, sheet_name="Trabalho")
    df_query_temp_aux.to_excel(writer, sheet_name="Auxílio")

# files.download('Temporal.xlsx')


### **ANÁLISE GEOGRÁFICA**
* Quantidade de testes
* Avaliar tipo de teste
* Providencias tomadas
* Trabalhou na semana anterior
* Recebeu auxílio
* Sintomas por região
* Escolaridade

In [30]:

df_query_geo = df_query.groupby(['UF'], as_index=False).agg({'total':'sum'})
df_query_geo = df_query_geo.pivot_table(index='UF', values='total')


df_query_geo_exam = df_query.groupby(['UF', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_geo_exam = df_query_geo_exam.pivot_table(index='UF', columns='fez_exame', values='total')

df_query_geo_result = df_query.groupby(['UF', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_geo_result = df_query_geo_result.pivot_table(index='UF', columns='resultado_final', values='total')


df_query_geo_trab = df_query.groupby(['UF', 'trabalhou_semana_passada'], as_index=False).agg({'total':'sum'})
df_query_geo_trab = df_query_geo_trab.pivot_table(index='UF', columns='trabalhou_semana_passada', values='total')

df_query_geo_aux = df_query.groupby(['UF', 'recebeu_auxilio'], as_index=False).agg({'total':'sum'})
df_query_geo_aux = df_query_geo_aux.pivot_table(index='UF', columns='recebeu_auxilio', values='total')

df_query_geo_esc = df_query.groupby(['UF', 'escolaridade'], as_index=False).agg({'total':'sum'})
df_query_geo_esc = df_query_geo_esc.pivot_table(index='UF', columns='escolaridade', values='total')

with pd.ExcelWriter("Geografica.xlsx", engine="openpyxl") as writer:
    df_query_geo.to_excel(writer, sheet_name="Total por Mês")
    df_query_geo_exam.to_excel(writer, sheet_name="Fez Exame")
    df_query_geo_result.to_excel(writer, sheet_name="Resultados Exame")
    df_query_geo_trab.to_excel(writer, sheet_name="Trabalho")
    df_query_geo_aux.to_excel(writer, sheet_name="Auxílio")
    df_query_geo_esc.to_excel(writer, sheet_name="Escolaridade")

# files.download('Geografica.xlsx')


### **ANÁLISE POR CARACTERÍSTICA**
* Sexo e idade
* Quantidade de testes
* Avaliar tipo de teste
* Providencias tomadas
* Trabalhou na semana anterior
* Sintomas por caracteristica


In [65]:
df_query_faixa = df_query.copy()

df_query_faixa['idade'] = df_query_faixa['idade'].apply(lambda x: '0-9' if x < 10 else
                                                                  '10-19' if x < 20 else
                                                                  '20-29' if x < 30 else
                                                                  '30-39' if x < 40 else
                                                                  '40-49' if x < 50 else
                                                                  '50-59' if x < 60 else
                                                                  '60-69' if x < 70 else
                                                                  '70-79' if x < 80 else
                                                                  '80-89' if x < 90 else
                                                                  '90-99' if x < 100 else '100+')

df_query_pop = df_query_faixa.groupby(['idade'], as_index=False).agg({'total':'sum'})

df_query_pop_exam = df_query_faixa.groupby(['idade', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_pop_exam = df_query_pop_exam.pivot_table(index='idade', columns='fez_exame', values='total')

df_query_pop_result = df_query_faixa.groupby(['idade', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_pop_result = df_query_pop_result.pivot_table(index='idade', columns='resultado_final', values='total')

df_query_pop_trab = df_query_faixa.groupby(['idade', 'trabalhou_semana_passada'], as_index=False).agg({'total':'sum'})
df_query_pop_trab = df_query_pop_trab.pivot_table(index='idade', columns='trabalhou_semana_passada', values='total')


df_query_pop_aux = df_query_faixa.groupby(['idade', 'recebeu_auxilio'], as_index=False).agg({'total':'sum'})
df_query_pop_aux = df_query_pop_aux.pivot_table(index='idade', columns='recebeu_auxilio', values='total')

df_query_pop_esc = df_query_faixa.groupby(['idade', 'escolaridade'], as_index=False).agg({'total':'sum'})
df_query_pop_esc = df_query_pop_esc.pivot_table(index='idade', columns='escolaridade', values='total')

df_query_pop_sexo = df_query_faixa.groupby(['sexo'], as_index=False).agg({'total':'sum'})

df_query_pop_exam_sexo = df_query_faixa.groupby(['sexo', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_pop_exam_sexo = df_query_pop_exam_sexo.pivot_table(index='sexo', columns='fez_exame', values='total')

df_query_pop_result_sexo = df_query_faixa.groupby(['sexo', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_pop_result_sexo = df_query_pop_result_sexo.pivot_table(index='sexo', columns='resultado_final', values='total')

with pd.ExcelWriter("Populacao.xlsx", engine="openpyxl") as writer:
    df_query_pop.to_excel(writer, sheet_name="Total por Mês")
    df_query_pop_exam.to_excel(writer, sheet_name="Fez Exame")
    df_query_pop_result.to_excel(writer, sheet_name="Resultados Exame")
    df_query_pop_trab.to_excel(writer, sheet_name="Trabalho")
    df_query_pop_aux.to_excel(writer, sheet_name="Auxílio")
    df_query_pop_esc.to_excel(writer, sheet_name="Escolaridade")
    df_query_pop_sexo.to_excel(writer, sheet_name="Sexo")
    df_query_pop_exam_sexo.to_excel(writer, sheet_name="Exame_sexo")
    df_query_pop_result_sexo.to_excel(writer, sheet_name="resul_sexo")

files.download('Populacao.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [68]:

df_query_eco = df_query.groupby(['recebeu_auxilio'], as_index=False).agg({'total':'sum'})

df_query_eco_exam = df_query.groupby(['recebeu_auxilio', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_eco_exam = df_query_eco_exam.pivot_table(index='recebeu_auxilio', columns='fez_exame', values='total')

df_query_eco_result = df_query.groupby(['recebeu_auxilio', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_eco_result = df_query_eco_result.pivot_table(index='recebeu_auxilio', columns='resultado_final', values='total')

df_query_eco_trab = df_query.groupby(['recebeu_auxilio', 'trabalhou_semana_passada'], as_index=False).agg({'total':'sum'})
df_query_eco_trab = df_query_eco_trab.pivot_table(index='recebeu_auxilio', columns='trabalhou_semana_passada', values='total')


df_query_eco_esc = df_query.groupby(['recebeu_auxilio', 'escolaridade'], as_index=False).agg({'total':'sum'})
df_query_eco_esc = df_query_eco_esc.pivot_table(index='recebeu_auxilio', columns='escolaridade', values='total')

df_query_eco_escolaridade = df_query.groupby(['escolaridade'], as_index=False).agg({'total':'sum'})

df_query_eco_exam_escolaridade = df_query.groupby(['escolaridade', 'fez_exame'], as_index=False).agg({'total':'sum'})
df_query_eco_exam_escolaridade = df_query_eco_exam_escolaridade.pivot_table(index='escolaridade', columns='fez_exame', values='total')

df_query_eco_result_escolaridade = df_query.groupby(['escolaridade', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_eco_result_escolaridade = df_query_eco_result_escolaridade.pivot_table(index='escolaridade', columns='resultado_final', values='total')

with pd.ExcelWriter("Economia.xlsx", engine="openpyxl") as writer:
    df_query_eco.to_excel(writer, sheet_name="Total por Mês")
    df_query_eco_exam.to_excel(writer, sheet_name="Fez Exame")
    df_query_eco_result.to_excel(writer, sheet_name="Resultados Exame")
    df_query_eco_trab.to_excel(writer, sheet_name="Trabalho")
    df_query_eco_esc.to_excel(writer, sheet_name="Escolaridade")
    df_query_eco_escolaridade.to_excel(writer, sheet_name="escolaridade")
    df_query_eco_exam_escolaridade.to_excel(writer, sheet_name="Exame_escolaridade")
    df_query_eco_result_escolaridade.to_excel(writer, sheet_name="resul_escolaridade")

files.download('Economia.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
sintoma_cols = [col for col in df_query_faixa.columns if col.startswith('teve_')]

df_melt = df_query_faixa.melt(
    id_vars=['fez_exame', 'resultado_final', 'providencia_exame', 'total', 'idade'],
    value_vars=sintoma_cols,
    var_name='sintoma',
    value_name='teve'
)


df_melt['sintoma'] = df_melt['sintoma'].str.replace('teve_', '')

df_resumo = df_melt.groupby(
    ['sintoma', 'fez_exame', 'resultado_final', 'providencia_exame', 'idade']
)['total'].sum().reset_index()


df_query_sintoma_result = df_resumo.groupby(['sintoma', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_sintoma_result = df_query_sintoma_result.pivot_table(index='sintoma', columns='resultado_final', values='total')

df_query_sintoma_idade = df_resumo.groupby(['sintoma', 'idade'], as_index=False).agg({'total':'sum'})
df_query_sintoma_idade = df_query_sintoma_idade.pivot_table(index='sintoma', columns='idade', values='total')

df_query_sintoma_prov = df_resumo.groupby(['providencia_exame', 'resultado_final'], as_index=False).agg({'total':'sum'})
df_query_sintoma_prov = df_query_sintoma_prov.pivot_table(index='providencia_exame', columns='resultado_final', values='total')

df_query_sintoma = df_resumo.groupby(['sintoma'], as_index=False).agg({'total':'sum'})


with pd.ExcelWriter("Sintoma.xlsx", engine="openpyxl") as writer:
    df_query_sintoma.to_excel(writer, sheet_name="sintoma")
    df_query_sintoma_result.to_excel(writer, sheet_name="result")
    df_query_sintoma_prov.to_excel(writer, sheet_name="prov")
    df_query_sintoma_idade.to_excel(writer, sheet_name="idade")


files.download('Sintoma.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>